In [1]:
using LinearAlgebra
using StaticArrays


In [2]:
#import Pkg; Pkg.add("StaticArrays")

In [3]:
SVector{4,Float32}

SArray{Tuple{4},Float32,1,4}

In [4]:
d = 3
c = 1
key_type = SVector{d,Float32}
val_type = SVector{c,Float32}
lattice = Dict{key_type,val_type}

Dict{SArray{Tuple{3},Float32,1,3},SArray{Tuple{1},Float32,1,1}}

In [52]:
function canonical_vertices(d)
    canonical_coords = Array{Float32}(undef,d+1,d+1)
    for k in 1:d+1
        canonical_coords[k,1:d+2-k].=k-1
        canonical_coords[k,d-k+3:end].=k-d-2
    end
    return canonical_coords'
end
function elevate(x::Array{Float32,1})
    d= length(x)
    s = [(d+1)/√(i*(i+1)*2/3) for i in 1:d+1] # Scale factor
    elevated = zeros(Float32,d)
    elevated[d] = -d*x[d]s[d];
    for i in d-1:-1:2
        elevated[i] = elevated[i + 1] - (i+1)*x[i-1]s[i-1] + (i + 3)*x[i]s[i]
    end
    elevated[1] = elevated[1] + 2x[1]s[1];
    return elevated
end

function elevate(x::Array{Float32,2})
    return [elevate(xi) for xi in x]
end

function closest_0cell(Ex::Array{Float32,1})
    d = length(Ex)
    mygreedy = zeros(Float32,d+1)
    for i in 1:d+1
        v = Ex[i]/(d+1)
        up = ceil(v)*(d+1);
        down = floor(v)*(d+1);
        if (up - Ex[i] < Ex[i] - down)
            mygreedy[i] = up
        else
            mygreedy[i] = down
        end
    end
    return mygreedy
end

function closest_0cell(Ex::Array{Float32,2})
    return [closest_0cell(xi) for xi in Ex]
end

function ranks(Ex::Array{Float32,1},closest0::Array{Float32,1})
    d = length(Ex)
    theranks = zeros(Int,d+1)
    coordsum = sum(closest0)/(d+1)
    for i in 1:d
        for j in i+1:d+1
            if (Ex[i] - closest0[i] < Ex[j] - closest0[j])
                theranks[i]+=1
            else
                theranks[j]+=1
            end
        end
    end
    if (coordsum > 0)
        #sum too large - the point is off the hyperplane.
        #need to bring down the ones with the smallest differential
        for i in 1:d+1
            if (myrank[i] >= d + 1 - coordsum)
                mygreedy[i] -= d + 1
                myrank[i] += coordsum - (d + 1)
            else
                myrank[i] += coordsum
            end
        end
    elseif (coordsum < 0)
        #sum too small - the point is off the hyperplane
        #need to bring up the ones with largest differential
        for i in 1:d+1
            if (myrank[i] < -coordsum)
                mygreedy[i] += d + 1
                myrank[i] += (d + 1) + coordsum
            else
                myrank[i] += coordsum
            end
        end
    end
end

function ranks(Ex::Array{Float32,2},closest0::Array{Float32,2})
    return [ranks(xi) for xi in Ex]
end

function keys(X)
    EX = elevate(X)
    closest_vertex = closest_0cell(EX)
    permutations = ranks(EX,closest_vertex)
    n,d = size(X)
    barycentric_weights = zeros(n,d+2)
    for i in 1:d+1
        barycentric_weights[d-myrank[i]] += (EX)
        
    ...
    
end
function splat!(lattice,coords,vals)
    ...
end

2×5 Array{Float32,2}:
  4.5605f-41  -9.85772f-26   4.5605f-41   -2.05045f-26   4.5605f-41
 -9.8577f-26   4.5605f-41   -2.05043f-26   4.5605f-41   -2.05054f-26

In [103]:
[x for x in rand(3,4)]

3×4 Array{Float64,2}:
 0.592251  0.131684   0.0887873  0.427838
 0.243746  0.335925   0.50437    0.190674
 0.128383  0.0840014  0.0944718  0.125596

In [112]:
zeros(Float32,5)

5-element Array{Float32,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [102]:
[(i+1)(i+1) for i in 4:-1:1]

LoadError: MethodError: objects of type Int64 are not callable

In [62]:
canonical_coords'

5×5 Adjoint{Float32,Array{Float32,2}}:
 0.0   1.0   2.0   3.0   4.0
 0.0   1.0   2.0   3.0  -1.0
 0.0   1.0   2.0  -2.0  -1.0
 0.0   1.0  -3.0  -2.0  -1.0
 0.0  -4.0  -3.0  -2.0  -1.0

In [82]:
1,...,3

LoadError: syntax: invalid identifier name "..."

In [74]:
sum(1:20)

210